# CSC 786 – EDR Evasion Analysis: io_uring vs Traditional Syscalls

**Author:** Michael Mendoza  
**Course:** CSC 786 – Computer Science Problems  
**Institution:** Dakota State University

---

## Overview

This notebook analyzes experimental results comparing EDR visibility between:
- **Traditional syscall binaries** (baseline)
- **io_uring-based binaries** (evasion variant)

## Metrics Evaluated

### Primary Metrics
1. **Detection Rate** – Proportion of runs that produced at least one audit hit or Wazuh alert
2. **False Negative Rate** – Proportion of runs where behavior occurred but no alert was generated
3. **Mean Audit Hits** – Average syscall-related audit events per run

### Secondary Metrics
4. **Signal Fidelity** – Presence of contextual information in detections
5. **Evasion Delta** – Difference in detection rates between traditional and io_uring

## MITRE ATT&CK Mapping
- **T1059** – Command and Scripting Interpreter (exec_cmd)
- **T1071** – Application Layer Protocol (net_connect)
- **T1005** – Data from Local System (file_io, read_file)
- **T1562.001** – Impair Defenses: Disable or Modify Tools (io_uring evasion)

---

## 0) Setup and Configuration

In [ ]:
# Install dependencies if needed (uncomment if running fresh)
# !pip install pandas matplotlib seaborn scipy tabulate

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# Set style for publication-quality figures
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Create output directories
RESULTS_DIR = Path('results')
FIGURES_DIR = RESULTS_DIR / 'figures'
TABLES_DIR = RESULTS_DIR / 'tables'

for d in [RESULTS_DIR, FIGURES_DIR, TABLES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print('Output directories created:')
print(f'  Figures: {FIGURES_DIR}')
print(f'  Tables:  {TABLES_DIR}')

## 1) Load Experimental Data

In [ ]:
# Find the most recent CSV file
processed_dir = Path('../data/processed')

# Handle running from different directories
if not processed_dir.exists():
    processed_dir = Path('data/processed')
if not processed_dir.exists():
    processed_dir = Path('../data/processed')

csvs = sorted(processed_dir.glob('runs_*.csv'))

if not csvs:
    raise FileNotFoundError(
        f'No runs_*.csv found in {processed_dir}\n'
        'Run the experiment first: sudo ./run_experiment.sh 30'
    )

# Use the most recent CSV
csv_path = csvs[-1]
print(f'Loading: {csv_path}')
print(f'Available CSV files: {len(csvs)}')

# Load data
df = pd.read_csv(csv_path)

print(f'\nDataset shape: {df.shape[0]} rows × {df.shape[1]} columns')
print(f'Columns: {list(df.columns)}')
df.head(10)

## 2) Data Validation and Preprocessing

In [ ]:
# Validate required columns
required_cols = ['iteration', 'case', 'file_hits', 'net_hits', 'exec_hits', 'wazuh_alerts']
missing = [c for c in required_cols if c not in df.columns]

if missing:
    print(f'WARNING: Missing columns: {missing}')
    print('Some analyses may be limited.')
else:
    print('✓ All required columns present')

# Basic stats
print(f'\nIterations: {df["iteration"].nunique()}')
print(f'Test cases: {df["case"].nunique()}')
print(f'\nCases in dataset:')
for case in sorted(df['case'].unique()):
    count = len(df[df['case'] == case])
    print(f'  - {case}: {count} runs')

In [ ]:
# Add derived columns for analysis

# Classify each case as traditional or io_uring
def classify_case(case_name):
    if 'uring' in case_name.lower():
        return 'io_uring'
    else:
        return 'traditional'

df['method'] = df['case'].apply(classify_case)

# Classify by operation type
def classify_operation(case_name):
    case_lower = case_name.lower()
    if 'file_io' in case_lower:
        return 'File I/O (write)'
    elif 'read_file' in case_lower or 'openat' in case_lower:
        return 'File I/O (read)'
    elif 'net' in case_lower:
        return 'Network'
    elif 'exec' in case_lower:
        return 'Process Exec'
    else:
        return 'Other'

df['operation'] = df['case'].apply(classify_operation)

# Calculate total audit hits (sum of all audit sources)
df['total_audit_hits'] = df['file_hits'] + df['net_hits'] + df['exec_hits']

# Detection flags (binary: was it detected?)
df['audit_detected'] = df['total_audit_hits'] > 0
df['wazuh_detected'] = df['wazuh_alerts'] > 0
df['any_detected'] = df['audit_detected'] | df['wazuh_detected']

print('Derived columns added:')
print('  - method: traditional vs io_uring')
print('  - operation: File I/O, Network, Process Exec')
print('  - total_audit_hits: sum of all audit hits')
print('  - audit_detected, wazuh_detected, any_detected: boolean flags')

df[['case', 'method', 'operation', 'total_audit_hits', 'audit_detected']].head(10)

## 3) Primary Metric: Detection Rate

**Definition:** Proportion of runs that produced at least one audit hit or Wazuh alert.

This directly answers: *Did the EDR see the activity?*

In [ ]:
# Calculate detection rates by case
detection_rates = df.groupby('case').agg({
    'audit_detected': 'mean',
    'wazuh_detected': 'mean',
    'any_detected': 'mean',
    'iteration': 'count'
}).rename(columns={
    'audit_detected': 'Audit Detection Rate',
    'wazuh_detected': 'Wazuh Detection Rate', 
    'any_detected': 'Overall Detection Rate',
    'iteration': 'N (runs)'
})

# Add method classification
detection_rates['Method'] = detection_rates.index.map(classify_case)

# Format as percentages for display
detection_rates_display = detection_rates.copy()
for col in ['Audit Detection Rate', 'Wazuh Detection Rate', 'Overall Detection Rate']:
    detection_rates_display[col] = (detection_rates_display[col] * 100).round(1).astype(str) + '%'

print('=== Detection Rates by Test Case ===')
print(detection_rates_display.to_string())

# Save to CSV
detection_rates.to_csv(TABLES_DIR / 'detection_rates_by_case.csv')
print(f'\nSaved: {TABLES_DIR}/detection_rates_by_case.csv')

In [ ]:
# Detection rates by method (traditional vs io_uring)
method_rates = df.groupby('method').agg({
    'audit_detected': ['mean', 'sum', 'count'],
    'wazuh_detected': ['mean', 'sum'],
}).round(4)

method_rates.columns = ['Audit Rate', 'Audit Detections', 'Total Runs', 'Wazuh Rate', 'Wazuh Detections']

print('=== Detection Rates: Traditional vs io_uring ===')
print(method_rates.to_string())

# Calculate evasion effectiveness
if 'traditional' in method_rates.index and 'io_uring' in method_rates.index:
    trad_rate = method_rates.loc['traditional', 'Audit Rate']
    uring_rate = method_rates.loc['io_uring', 'Audit Rate']
    evasion_delta = trad_rate - uring_rate
    evasion_pct = (evasion_delta / trad_rate * 100) if trad_rate > 0 else 0
    
    print(f'\n=== Evasion Effectiveness ===')
    print(f'Traditional detection rate: {trad_rate:.1%}')
    print(f'io_uring detection rate:    {uring_rate:.1%}')
    print(f'Detection reduction:        {evasion_delta:.1%} ({evasion_pct:.1f}% evasion)')

## 4) Primary Metric: False Negative Rate

**Definition:** Proportion of runs where behavior occurred but no alert was generated.

For io_uring cases, we *expect* high false negatives (the evasion is working).  
For traditional cases, false negatives indicate gaps in audit rules.

In [ ]:
# False negative = behavior occurred but not detected
# Since all runs execute the behavior, FN rate = 1 - detection rate

fn_rates = df.groupby('case').agg({
    'audit_detected': lambda x: 1 - x.mean(),
    'wazuh_detected': lambda x: 1 - x.mean(),
}).rename(columns={
    'audit_detected': 'Audit FN Rate',
    'wazuh_detected': 'Wazuh FN Rate'
})

fn_rates['Method'] = fn_rates.index.map(classify_case)
fn_rates['Operation'] = fn_rates.index.map(classify_operation)

print('=== False Negative Rates by Test Case ===')
print('(Higher = more evasion / less visibility)\n')

fn_display = fn_rates.copy()
fn_display['Audit FN Rate'] = (fn_display['Audit FN Rate'] * 100).round(1).astype(str) + '%'
fn_display['Wazuh FN Rate'] = (fn_display['Wazuh FN Rate'] * 100).round(1).astype(str) + '%'
print(fn_display.to_string())

fn_rates.to_csv(TABLES_DIR / 'false_negative_rates.csv')
print(f'\nSaved: {TABLES_DIR}/false_negative_rates.csv')

## 5) Validation: Confirming io_uring Bypasses Syscall Traces
### Validation Approach

We validate bypass by comparing **audit hit counts** between paired operations:

| Operation | Traditional Binary | io_uring Binary | Expected Result |
|-----------|-------------------|-----------------|------------------|
| File Write | file_io_traditional | file_io_uring | Trad >> Uring |
| File Read | read_file_traditional | openat_uring | Trad >> Uring |
| Network | net_connect_traditional | net_connect_uring | Trad >> Uring |

If io_uring successfully bypasses syscall tracing:
1. Traditional binaries should generate **significantly more** audit hits
2. io_uring binaries should generate **near-zero** audit hits for file/net operations
3. The difference should be **statistically significant**

In [ ]:
# Define paired comparisons
pairs = [
    ('file_io_traditional', 'file_io_uring', 'file_hits', 'File Write'),
    ('read_file_traditional', 'openat_uring', 'file_hits', 'File Read'),
    ('net_connect_traditional', 'net_connect_uring', 'net_hits', 'Network Connect'),
]

validation_results = []

print('=== Syscall Bypass Validation ===' )
print('Comparing audit hits between traditional and io_uring binaries\n')

for trad_case, uring_case, metric_col, op_name in pairs:
    trad_data = df[df['case'] == trad_case][metric_col]
    uring_data = df[df['case'] == uring_case][metric_col]
    
    if len(trad_data) == 0 or len(uring_data) == 0:
        print(f'{op_name}: SKIPPED (missing data)')
        continue
    
    trad_mean = trad_data.mean()
    uring_mean = uring_data.mean()
    
    # Statistical test (Mann-Whitney U for non-normal distributions)
    stat, p_value = stats.mannwhitneyu(trad_data, uring_data, alternative='greater')
    
    # Effect size (difference in means)
    effect = trad_mean - uring_mean
    
    # Bypass confirmed if uring has significantly fewer hits
    bypass_confirmed = p_value < 0.05 and effect > 0
    
    result = {
        'Operation': op_name,
        'Traditional Mean': round(trad_mean, 2),
        'io_uring Mean': round(uring_mean, 2),
        'Difference': round(effect, 2),
        'p-value': f'{p_value:.4f}',
        'Bypass Confirmed': '✓ YES' if bypass_confirmed else '✗ NO'
    }
    validation_results.append(result)
    
    print(f'--- {op_name} ---')
    print(f'  Traditional ({trad_case}): mean = {trad_mean:.2f} hits')
    print(f'  io_uring ({uring_case}):    mean = {uring_mean:.2f} hits')
    print(f'  Difference: {effect:.2f} (p = {p_value:.4f})')
    print(f'  Syscall bypass confirmed: {"YES ✓" if bypass_confirmed else "NO ✗"}')
    print()

# Create validation summary table
validation_df = pd.DataFrame(validation_results)
validation_df.to_csv(TABLES_DIR / 'syscall_bypass_validation.csv', index=False)
print(f'Saved: {TABLES_DIR}/syscall_bypass_validation.csv')

## 6) Descriptive Statistics: Mean Audit Hits

In [ ]:
# Comprehensive summary statistics
metric_cols = ['file_hits', 'net_hits', 'exec_hits', 'total_audit_hits', 'wazuh_alerts']

summary_stats = df.groupby('case')[metric_cols].agg(['mean', 'std', 'min', 'max']).round(2)

print('=== Descriptive Statistics by Test Case ===')
print(summary_stats.to_string())

summary_stats.to_csv(TABLES_DIR / 'descriptive_statistics.csv')
print(f'\nSaved: {TABLES_DIR}/descriptive_statistics.csv')

In [ ]:
# Simplified mean comparison table (good for papers)
means_table = df.groupby('case')[metric_cols].mean().round(2)
means_table['Method'] = means_table.index.map(classify_case)
means_table['Operation'] = means_table.index.map(classify_operation)

# Reorder columns
means_table = means_table[['Method', 'Operation', 'file_hits', 'net_hits', 'exec_hits', 'total_audit_hits', 'wazuh_alerts']]

print('=== Mean Values by Test Case (for publication) ===')
print(means_table.to_string())

means_table.to_csv(TABLES_DIR / 'mean_hits_by_case.csv')
print(f'\nSaved: {TABLES_DIR}/mean_hits_by_case.csv')

## 7) Visualizations

In [ ]:
# Figure 1: Detection Rate Comparison (Bar Chart)
fig, ax = plt.subplots(figsize=(12, 6))

# Prepare data
plot_data = df.groupby(['case', 'method'])['audit_detected'].mean().reset_index()
plot_data['audit_detected'] *= 100  # Convert to percentage

# Create grouped bar chart
cases = plot_data['case'].unique()
x = np.arange(len(cases))
colors = ['#2ecc71' if 'traditional' in c else '#e74c3c' for c in cases]

bars = ax.bar(x, plot_data.groupby('case')['audit_detected'].first(), color=colors, edgecolor='black', linewidth=0.5)

ax.set_xlabel('Test Case')
ax.set_ylabel('Audit Detection Rate (%)')
ax.set_title('Figure 1: Audit Detection Rate by Test Case\n(Traditional vs io_uring)', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(cases, rotation=45, ha='right')
ax.set_ylim(0, 105)

# Add value labels on bars
for bar, val in zip(bars, plot_data.groupby('case')['audit_detected'].first()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val:.0f}%', 
            ha='center', va='bottom', fontsize=9)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#2ecc71', label='Traditional (syscalls)'),
                   Patch(facecolor='#e74c3c', label='io_uring (evasion)')]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig1_detection_rate_comparison.png', dpi=300, bbox_inches='tight')
plt.savefig(FIGURES_DIR / 'fig1_detection_rate_comparison.pdf', bbox_inches='tight')
plt.show()
print(f'Saved: {FIGURES_DIR}/fig1_detection_rate_comparison.png')

In [ ]:
# Figure 2: Mean Audit Hits Comparison (Grouped Bar Chart)
fig, ax = plt.subplots(figsize=(10, 6))

# Prepare data for paired comparison
pair_labels = ['File Write', 'File Read', 'Network']
trad_cases = ['file_io_traditional', 'read_file_traditional', 'net_connect_traditional']
uring_cases = ['file_io_uring', 'openat_uring', 'net_connect_uring']

trad_means = [df[df['case'] == c]['total_audit_hits'].mean() for c in trad_cases]
uring_means = [df[df['case'] == c]['total_audit_hits'].mean() for c in uring_cases]

x = np.arange(len(pair_labels))
width = 0.35

bars1 = ax.bar(x - width/2, trad_means, width, label='Traditional', color='#2ecc71', edgecolor='black')
bars2 = ax.bar(x + width/2, uring_means, width, label='io_uring', color='#e74c3c', edgecolor='black')

ax.set_xlabel('Operation Type')
ax.set_ylabel('Mean Audit Hits')
ax.set_title('Figure 2: Mean Audit Hits – Traditional vs io_uring\n(Lower io_uring values indicate successful syscall bypass)', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(pair_labels)
ax.legend()

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height + 0.5, f'{height:.1f}',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig2_mean_hits_comparison.png', dpi=300, bbox_inches='tight')
plt.savefig(FIGURES_DIR / 'fig2_mean_hits_comparison.pdf', bbox_inches='tight')
plt.show()
print(f'Saved: {FIGURES_DIR}/fig2_mean_hits_comparison.png')

In [ ]:
# Figure 3: Box Plot Distribution
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

comparisons = [
    ('file_io_traditional', 'file_io_uring', 'file_hits', 'File Write Operations'),
    ('read_file_traditional', 'openat_uring', 'file_hits', 'File Read Operations'),
    ('net_connect_traditional', 'net_connect_uring', 'net_hits', 'Network Operations'),
]

for ax, (trad, uring, metric, title) in zip(axes, comparisons):
    trad_data = df[df['case'] == trad][metric]
    uring_data = df[df['case'] == uring][metric]
    
    bp = ax.boxplot([trad_data, uring_data], labels=['Traditional', 'io_uring'],
                    patch_artist=True)
    
    bp['boxes'][0].set_facecolor('#2ecc71')
    bp['boxes'][1].set_facecolor('#e74c3c')
    
    ax.set_title(title)
    ax.set_ylabel('Audit Hits')

fig.suptitle('Figure 3: Distribution of Audit Hits by Operation Type', fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig3_boxplot_distributions.png', dpi=300, bbox_inches='tight')
plt.savefig(FIGURES_DIR / 'fig3_boxplot_distributions.pdf', bbox_inches='tight')
plt.show()
print(f'Saved: {FIGURES_DIR}/fig3_boxplot_distributions.png')

In [ ]:
# Figure 4: Heatmap of Detection Rates
fig, ax = plt.subplots(figsize=(10, 6))

# Prepare heatmap data
heatmap_data = df.groupby('case')[['file_hits', 'net_hits', 'exec_hits', 'wazuh_alerts']].apply(
    lambda x: (x > 0).mean()
) * 100

heatmap_data.columns = ['File Audit', 'Network Audit', 'Exec Audit', 'Wazuh Alerts']

# Sort by method for better visualization
heatmap_data['sort_key'] = heatmap_data.index.map(lambda x: 0 if 'traditional' in x else 1)
heatmap_data = heatmap_data.sort_values('sort_key').drop('sort_key', axis=1)

sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='RdYlGn', 
            linewidths=0.5, ax=ax, vmin=0, vmax=100,
            cbar_kws={'label': 'Detection Rate (%)'})

ax.set_title('Figure 4: Detection Rate Heatmap by Test Case and Metric\n(Green = High Detection, Red = Low/Evasion)', 
             fontweight='bold')
ax.set_xlabel('Detection Source')
ax.set_ylabel('Test Case')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig4_detection_heatmap.png', dpi=300, bbox_inches='tight')
plt.savefig(FIGURES_DIR / 'fig4_detection_heatmap.pdf', bbox_inches='tight')
plt.show()
print(f'Saved: {FIGURES_DIR}/fig4_detection_heatmap.png')

In [ ]:
# Figure 5: Evasion Effectiveness Summary
fig, ax = plt.subplots(figsize=(8, 6))

# Calculate evasion rates by operation
operations = ['File Write', 'File Read', 'Network']
trad_detection = []
uring_detection = []

for trad, uring in [('file_io_traditional', 'file_io_uring'), 
                    ('read_file_traditional', 'openat_uring'),
                    ('net_connect_traditional', 'net_connect_uring')]:
    trad_rate = df[df['case'] == trad]['audit_detected'].mean() * 100
    uring_rate = df[df['case'] == uring]['audit_detected'].mean() * 100
    trad_detection.append(trad_rate)
    uring_detection.append(uring_rate)

evasion_pct = [t - u for t, u in zip(trad_detection, uring_detection)]

colors = ['#3498db' if e > 0 else '#95a5a6' for e in evasion_pct]
bars = ax.barh(operations, evasion_pct, color=colors, edgecolor='black')

ax.set_xlabel('Detection Rate Reduction (%)')
ax.set_title('Figure 5: io_uring Evasion Effectiveness\n(Reduction in Detection Rate vs Traditional)', fontweight='bold')
ax.axvline(x=0, color='black', linewidth=0.5)

# Add value labels
for bar, val in zip(bars, evasion_pct):
    ax.text(val + 1, bar.get_y() + bar.get_height()/2, f'{val:.1f}%',
            va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig5_evasion_effectiveness.png', dpi=300, bbox_inches='tight')
plt.savefig(FIGURES_DIR / 'fig5_evasion_effectiveness.pdf', bbox_inches='tight')
plt.show()
print(f'Saved: {FIGURES_DIR}/fig5_evasion_effectiveness.png')

## 8) Statistical Significance Testing

In [ ]:
# Perform statistical tests for all paired comparisons
print('=== Statistical Significance Tests ===')
print('H₀: Traditional and io_uring have equal audit hit distributions')
print('H₁: Traditional has greater audit hits than io_uring')
print('Test: Mann-Whitney U (one-tailed, α = 0.05)\n')

stat_results = []

test_pairs = [
    ('file_io_traditional', 'file_io_uring', 'file_hits'),
    ('read_file_traditional', 'openat_uring', 'file_hits'),
    ('net_connect_traditional', 'net_connect_uring', 'net_hits'),
]

for trad_case, uring_case, metric in test_pairs:
    trad = df[df['case'] == trad_case][metric]
    uring = df[df['case'] == uring_case][metric]
    
    if len(trad) == 0 or len(uring) == 0:
        continue
    
    stat, p = stats.mannwhitneyu(trad, uring, alternative='greater')
    
    # Effect size (rank-biserial correlation)
    n1, n2 = len(trad), len(uring)
    effect_size = 1 - (2*stat)/(n1*n2)
    
    significant = p < 0.05
    
    stat_results.append({
        'Comparison': f'{trad_case} vs {uring_case}',
        'Metric': metric,
        'U-statistic': stat,
        'p-value': p,
        'Effect Size (r)': round(effect_size, 3),
        'Significant': 'Yes' if significant else 'No'
    })
    
    print(f'{trad_case} vs {uring_case}:')
    print(f'  U = {stat:.1f}, p = {p:.6f}, r = {effect_size:.3f}')
    print(f'  Result: {"SIGNIFICANT ✓" if significant else "Not significant"}')
    print()

stat_df = pd.DataFrame(stat_results)
stat_df.to_csv(TABLES_DIR / 'statistical_tests.csv', index=False)
print(f'Saved: {TABLES_DIR}/statistical_tests.csv')

## 9) MITRE ATT&CK Mapping Table

In [ ]:
# Create MITRE ATT&CK mapping table
mitre_mapping = pd.DataFrame([
    {
        'Test Case': 'exec_cmd_traditional',
        'Operation': 'Process Execution',
        'MITRE Technique': 'T1059',
        'Technique Name': 'Command and Scripting Interpreter',
        'io_uring Variant': 'N/A (execve cannot use io_uring)',
        'Evasion Possible': 'No'
    },
    {
        'Test Case': 'file_io_traditional / file_io_uring',
        'Operation': 'File Write',
        'MITRE Technique': 'T1005',
        'Technique Name': 'Data from Local System',
        'io_uring Variant': 'IORING_OP_WRITE',
        'Evasion Possible': 'Yes'
    },
    {
        'Test Case': 'read_file_traditional / openat_uring',
        'Operation': 'File Read',
        'MITRE Technique': 'T1005',
        'Technique Name': 'Data from Local System',
        'io_uring Variant': 'IORING_OP_OPENAT + READ',
        'Evasion Possible': 'Yes'
    },
    {
        'Test Case': 'net_connect_traditional / net_connect_uring',
        'Operation': 'Network Connection',
        'MITRE Technique': 'T1071',
        'Technique Name': 'Application Layer Protocol',
        'io_uring Variant': 'IORING_OP_CONNECT',
        'Evasion Possible': 'Yes'
    },
    {
        'Test Case': 'All io_uring variants',
        'Operation': 'Defense Evasion',
        'MITRE Technique': 'T1562.001',
        'Technique Name': 'Impair Defenses: Disable or Modify Tools',
        'io_uring Variant': 'Syscall trace bypass',
        'Evasion Possible': 'Yes (primary finding)'
    },
])

print('=== MITRE ATT&CK Mapping ===')
print(mitre_mapping.to_string(index=False))

mitre_mapping.to_csv(TABLES_DIR / 'mitre_attack_mapping.csv', index=False)
print(f'\nSaved: {TABLES_DIR}/mitre_attack_mapping.csv')

## 10) Executive Summary

In [ ]:
# Generate executive summary
print('=' * 70)
print('EXECUTIVE SUMMARY: EDR Evasion via io_uring')
print('=' * 70)

# Overall statistics
total_runs = len(df)
iterations = df['iteration'].nunique()
cases = df['case'].nunique()

print(f'\nExperiment Overview:')
print(f'  Total test runs: {total_runs}')
print(f'  Iterations: {iterations}')
print(f'  Test cases: {cases}')

# Detection comparison
trad_df = df[df['method'] == 'traditional']
uring_df = df[df['method'] == 'io_uring']

print(f'\nDetection Rates:')
print(f'  Traditional syscalls: {trad_df["audit_detected"].mean():.1%} audit detection')
print(f'  io_uring operations:  {uring_df["audit_detected"].mean():.1%} audit detection')

# Key finding
evasion_rate = trad_df['audit_detected'].mean() - uring_df['audit_detected'].mean()
print(f'\nKey Finding:')
print(f'  io_uring reduces audit detection by {evasion_rate:.1%}')

print(f'\nConclusion:')
if evasion_rate > 0.5:
    print('  ⚠️  SIGNIFICANT EVASION GAP CONFIRMED')
    print('  io_uring operations successfully bypass syscall-based monitoring.')
    print('  Defenders should implement eBPF-based monitoring or audit io_uring_enter().')
elif evasion_rate > 0:
    print('  ⚠️  PARTIAL EVASION GAP DETECTED')
    print('  io_uring shows reduced visibility compared to traditional syscalls.')
else:
    print('  ✓ No significant evasion gap detected in this configuration.')

print(f'\nOutput Files:')
print(f'  Figures: {FIGURES_DIR}/')
print(f'  Tables:  {TABLES_DIR}/')
print('=' * 70)

In [ ]:
# List all generated output files
print('\n=== Generated Output Files ===')

print('\nFigures:')
for f in sorted(FIGURES_DIR.glob('*')):
    print(f'  {f.name}')

print('\nTables (CSV):')
for f in sorted(TABLES_DIR.glob('*.csv')):
    print(f'  {f.name}')

print('\nUse these files in your final report!')

---

## Notes for Final Report

### Key Evidence for Syscall Bypass Validation

The validation section (Section 5) provides statistical evidence that io_uring truly bypasses syscall traces:

1. **Quantitative difference**: Traditional binaries generate significantly more audit hits
2. **Statistical significance**: Mann-Whitney U tests confirm the difference is not due to chance
3. **Consistent pattern**: All three operation types (file write, file read, network) show the same pattern

### Figures for Publication

- **Figure 1**: Detection rate bar chart (main finding)
- **Figure 2**: Mean audit hits comparison (quantitative evidence)
- **Figure 3**: Box plots showing distributions (variance analysis)
- **Figure 4**: Heatmap (comprehensive view)
- **Figure 5**: Evasion effectiveness summary (executive summary visual)

### Tables for Publication

- `detection_rates_by_case.csv`: Primary detection rate metrics
- `syscall_bypass_validation.csv`: Evidence for bypass confirmation
- `statistical_tests.csv`: Statistical significance results
- `mitre_attack_mapping.csv`: MITRE ATT&CK technique mapping